Loading the necessary packages

In [ ]:
from __future__ import print_function

from depmapomics.config import *

from depmapomics import tracker, loading, fusions, expressions
from genepy import terra
from genepy.utils import helper as h

import dalmatian as dm
import seaborn as sns

from bokeh.plotting import output_notebook
import autoreload

%load_ext autoreload
%autoreload 2

output_notebook()

In [ ]:
a = pd.read_csv('temp/expression.20Q2.genes')

In [ ]:
a

# Generate sample set from new samples

we retrieve all the samples we can find from the GP workspaces

__CCLE specific__

In [ ]:
if isCCLE:
    print("loading new RNAseq data")
    rnasamples = loading.loadRNA(SAMPLESETNAME)

In [ ]:
LINES_TO_RELEASE = [
    "ACH-000023",
    "ACH-000145",
    "ACH-000205",
    "ACH-000345",
    "ACH-000359",
    "ACH-000399",
    "ACH-000409",
    "ACH-000515",
    "ACH-000664",
    "ACH-000744",
    "ACH-000992",
    "ACH-001032",
    "ACH-001373",
    "ACH-001393",
    "ACH-001410",
    "ACH-001558",
    "ACH-001679",
    "ACH-001696",
    "ACH-002022",
    "ACH-002060",
    "ACH-002659",
    "ACH-002687",
    "ACH-002707",
]

In [ ]:
if isCCLE:
    ref = tracker.getCCLETracker()
    print('samples without rnaseq:')
    print(set(LINES_TO_RELEASE) - (set(rnasamples.arxspan_id) | set(ref[ref.datatype=='rna'].arxspan_id)))
    %store rnasamples

In [ ]:
if isCCLE:
    %store -r rnasamples
    print("uploading samples to the tracker and Terra: "+SAMPLESETNAME)
    loading.update(rnasamples, samplesetname=SAMPLESETNAME, stype='rna', bucket="gs://cclebams/rna/", refworkspace=RNAWORKSPACE)

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 6 functions To generate the expression/fusion dataset:

We use the GTEx pipeline ([https://github.com/broadinstitute/gtex-pipeline/blob/v9/TOPMed_RNAseq_pipeline.md](https://github.com/broadinstitute/gtex-pipeline/blob/v9/TOPMed_RNAseq_pipeline.md)).

To generate the expression dataset, run the following tasks on all samples that you need, in this order:



*   samtofastq_v1-0_BETA_cfg 

    (broadinstitute_gtex/samtofastq_v1-0_BETA Snapshot ID: 5)

*   star_v1-0_BETA_cfg

(broadinstitute_gtex/star_v1-0_BETA Snapshot ID: 7)



*   rsem_v1-0_BETA_cfg 

    (broadinstitute_gtex/rsem_v1-0_BETA Snapshot ID: 4)

*   rsem_aggregate_results_v1-0_BETA_cfg (broadinstitute_gtex/rsem_aggregate_results_v1-0_BETA Snapshot ID: 3)

The outputs to be downloaded will be saved under the sample set that you ran. The outputs we use for the release are:



*   rsem_genes_expected_count
*   rsem_genes_tpm
*   rsem_transcripts_tpm

****Make sure that you delete the intermediate files. These files are quite large so cost a lot to store. To delete, you can either write a task that deletes them or use gsutil rm*****


##### Fusions {#fusions}

We use STAR-Fusion [https://github.com/STAR-Fusion/STAR-Fusion/wiki](https://github.com/STAR-Fusion/STAR-Fusion/wiki). The fusions are generated by running the following tasks



*   hg38_STAR_fusion (gkugener/STAR_fusion Snapshot ID: 14)
*   Aggregate_Fusion_Calls (gkugener/Aggregate_files_set Snapshot ID: 2)

The outputs to be downloaded will be saved under the sample set you ran. The outputs we use for the release are: 



*   fusions_star

This task uses the same samtofastq_v1-0_BETA_cfg task as in the expression pipeline, although in the current implementation, this task will be run twice. It might be worth combing the expression/fusion calling into a single workflow. This task also contains a flag that lets you specify if you want to delete the intermediates (fastqs). 

There are several other tasks in this workspace. In brief:



*   Tasks prefixed with **EXPENSIVE** or **CHEAP** are identical to their non-prefixed version, except that they specify different memory, disk space, etc. parameters. These versions can be used when samples fail the normal version of the task due to memory errors.
*   The following tasks are part of the GTEx pipeline but we do not use them (we use RSEM exclusively): markduplicates_v1-0_BETA_cfg (broadinstitute_gtex/markduplicates_v1-0_BETA Snapshot ID: 2), rnaseqc2_v1-0_BETA_cfg (broadinstitute_gtex/rnaseqc2_v1-0_BETA Snapshot ID: 2)
*   **ExonUsage_hg38_fixed** (gkugener/ExonUsage_fixed Snapshot ID: 1): this task calculates exon usage ratios. The non-fixed version contains a bug in the script that is not able to handle chromosome values prefixed with ‘chr’. The ‘fixed’ version resolves this issue.
*   **AggregateExonUsageRObj_hg38** (ccle_mg/AggregateExonUsageRObj Snapshot ID: 2): combines the exon usage ratios into a matrices that are saved in an R object.

## cleaning workspaces

In [ ]:
if doCleanup:
    print("cleaning workspaces")
    torm = await terra.deleteHeavyFiles(RNAWORKSPACE)
    h.parrun(['gsutil rm '+i for i in torm], cores=8)
    terra.removeFromFailedWorkflows(RNAWORKSPACE, dryrun=False, everythingFor=[])

## On Terra

In [ ]:
# TODO: update with latest workspace parameters from our repo

In [ ]:
print("running Terra pipeline")
refwm = dm.WorkspaceManager(RNAWORKSPACE)
submission_id = refwm.create_submission("RNA_pipeline", SAMPLESETNAME,'sample_set',expression='this.samples')
await terra.waitForSubmission(RNAWORKSPACE, submission_id)

In [ ]:
submission_id = refwm.create_submission("RNA_aggregate", 'all')
await terra.waitForSubmission(RNAWORKSPACE, submission_id)

### Save the workflow configurations used

In [ ]:
terra.saveWorkspace(RNAWORKSPACE,'data/'+SAMPLESETNAME+'/RNAconfig/')

## On Local

### Expression post processing

In [ ]:
await expressions.CCLEPostProcessing(samplesetname=SAMPLESETNAME, recompute_ssgsea=False)

In [ ]:
from depmapomics import tracker

In [ ]:
ref = tracker.getCCLETracker()

In [ ]:
#to set as isogenic
a = {
"ACH-001200": "ACH-000597",
"ACH-002061": "ACH-000551",
"ACH-000169": "ACH-001759",
"ACH-000169": "ACH-001758",
"ACH-000169": "ACH-001756",
"ACH-000169": "ACH-001760",
"ACH-002060": "ACH-002276",
"ACH-002238": "ACH-001669",
"ACH-001743": "ACH-002501",
"ACH-002162": "ACH-002084",
"ACH-001571": "ACH-002339",
}

In [ ]:
ref = tracker.updateIsogenecity(a, ref)

In [ ]:
ref.loc[ref[ref.arxspan_id=="ACH-001790"].index, ['stripped_cell_line_name','blacklist']] = ['RH18DMWRONG', 1]
ref.loc[ref[ref.arxspan_id=="ACH-002707"].index, ['stripped_cell_line_name','blacklist']] = ['IMSM2WRONG', 1]
ref.loc[ref[ref.arxspan_id=="ACH-001078"].index, ['stripped_cell_line_name','blacklist']] = ['HCC1588WRONG', 1]
ref.loc[ref[ref.arxspan_id=="ACH-002022"].index, ['stripped_cell_line_name','blacklist']] = ['PMFKO14WRONG', 1]
ref.loc[ref[ref.arxspan_id=="ACH-001173"].index, ['stripped_cell_line_name','blacklist']] = ['R262WRONG', 1]
ref.loc[ref[ref.arxspan_id=="ACH-001172"].index, ['stripped_cell_line_name','blacklist']] = ['R256WRONG', 1]

In [ ]:
# to unset as isogenic
di = {"ACH-001001": "ACH-000825",
"ACH-000717": "ACH-001547",
"ACH-002399": "ACH-001707",
"ACH-000590": "ACH-000681",
"ACH-001137": "ACH-002337",
"ACH-000957": "ACH-000925",
"ACH-001858": "ACH-001856",
"ACH-000167": "ACH-000658",
"ACH-000201": "ACH-001212",}

In [ ]:
ref

In [ ]:
tracker.updateIsogenecity(di, ref, unset=True)

# Fusion post processing

In [ ]:
await fusions.CCLEPostProcessing(samplesetname=SAMPLESETNAME)